<a href="https://colab.research.google.com/github/outofray/Explainable_Neural_Network/blob/main/Deep_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import library

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Attention
from tensorflow.keras.optimizers import SGD, Adam, Adagrad

# 2. Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 4. Split train and validation set

In [ ]:
#Splitting the data into independent and dependent variables
x = data.iloc[:,2:14]
y = data.iloc[:,16]

# Feature Scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x)

y_train = np.array(y)

print(x_train.shape)
print(y_train.shape)

In [ ]:
x_test = data_test.iloc[:,2:14]
y_test = data_test.iloc[:,16]

x_test = np.array(x_test, dtype=np.float)
y_test = np.array(y_test, dtype=np.float)

# Feature Scaling
x_test = scaler.fit_transform(x_test)

print(x_test.shape)
print(y_test.shape)

## K-fold cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

loss = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adamax(learning_rate=0.0002)
class_weight = {0: 1, 1: 100}

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
cvscores = []
aucscore = []
i=1

for train, val in kfold.split(x_train, y_train):
  #建構神經網路
  model = Sequential()
  model.add(Dense(32, input_dim=x_train.shape[1], activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  #最後要增加一層接回分類數，softmax方式可以讓所有的組合機率相加為一
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])

  checkpoint_filepath = "gdrive/MyDrive/IBI 60 days/IBI60/IBI60_best_weight_{}.hdf5".format(i)
  model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath, 
      save_best_only=True, 
      monitor='val_loss', #based on best val_loss
      mode='auto', verbose=0)
  callbacks = [model_checkpoint]

  model.fit(x_train[train], y_train[train], epochs=50, batch_size=32, class_weight=class_weight, callbacks=callbacks, verbose=0, shuffle=True)

  model.save(checkpoint_filepath)

  model.load_weights(checkpoint_filepath)

	# evaluate the model
  scores = model.evaluate(x_train[val], y_train[val], verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)

  y_val_prob = model.predict(x_train[val])
  print ('AUC value:{}'.format(roc_auc_score(y_train[val], y_val_prob)))
  aucscore.append(roc_auc_score(y_train[val], y_val_prob))

  i+=1

print("Mean accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("Mean AUC value: {:.2f} (+/- {:.2f})".format(np.mean(aucscore), np.std(aucscore)))